<a href="https://colab.research.google.com/github/vgaurav3011/EIP-3.0-/blob/master/Final_Assignment_3C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# https://keras.io/
!pip install -q keras
import keras

import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Reshape, Activation, Conv2D, Input, MaxPooling2D, BatchNormalization, Flatten, Dense, Lambda
from keras.layers.merge import concatenate

# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 10
num_filter = 20
compression = 0.5

In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoding 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

In [0]:
def add_block(input, num_filter = 20):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        x = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        x = Conv2D(int(32), (3, 3), use_bias = False, padding = 'same')(relu)
        x = Conv2D(int(64), (3, 3), use_bias = False, padding = 'same')(relu)
        x = Conv2D(int(128), (3, 3), use_bias = False, padding = 'same')(relu)
        x = Conv2D(int(256), (3, 3), use_bias = False, padding = 'same')(relu)
        x = Conv2D(int(512), (3, 3), use_bias = False, padding = 'same')(relu)
        
        concat = Concatenate(axis=-1)([temp,x])
        
        temp = concat
        
    return temp

In [0]:
def add_trans(input, num_filter = 20):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
def space_to_depth_x2(x):
    return tf.space_to_depth(x, block_size=2)

In [0]:
num_filter = 20
l = 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_block(First_Conv2D, num_filter)
First_Transition = add_trans(First_Block, num_filter)

Second_Block = add_block(First_Transition, num_filter)
Second_Transition = add_trans(Second_Block, num_filter)

Third_Block = add_block(Second_Transition, num_filter)
Third_Transition = add_trans(Third_Block, num_filter)

Fourth_Block = add_block(Third_Transition,  num_filter)
output = output_layer(Fourth_Block)

In [32]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_53 (Conv2D)              (None, 32, 32, 20)   540         input_8[0][0]                    
__________________________________________________________________________________________________
batch_normalization_53 (BatchNo (None, 32, 32, 20)   80          conv2d_53[0][0]                  
__________________________________________________________________________________________________
activation_53 (Activation)      (None, 32, 32, 20)   0           batch_normalization_53[0][0]     
__________________________________________________________________________________________________
conv2d_54 

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [34]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 180s 4ms/step - loss: 1.4545 - acc: 0.4669 - val_loss: 1.4870 - val_acc: 0.4805
Epoch 2/50
50000/50000 [==============================] - 157s 3ms/step - loss: 1.0447 - acc: 0.6253 - val_loss: 1.4468 - val_acc: 0.5152
Epoch 3/50
50000/50000 [==============================] - 157s 3ms/step - loss: 0.8664 - acc: 0.6911 - val_loss: 1.1894 - val_acc: 0.5982
Epoch 4/50
50000/50000 [==============================] - 158s 3ms/step - loss: 0.7459 - acc: 0.7351 - val_loss: 1.1462 - val_acc: 0.6042
Epoch 5/50
50000/50000 [==============================] - 157s 3ms/step - loss: 0.6584 - acc: 0.7654 - val_loss: 0.8778 - val_acc: 0.7000
Epoch 6/50
50000/50000 [==============================] - 157s 3ms/step - loss: 0.5926 - acc: 0.7918 - val_loss: 0.7641 - val_acc: 0.7360
Epoch 7/50
50000/50000 [==============================] - 157s 3ms/step - loss: 0.5369 - acc: 0.8094 - val_loss: 0.8744 - 

In [35]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
# Save the trained weights in to .h5 format
model.save_weights("Yolo_Basic_model2.h5")
print("Saved the model to disk")

10000/10000 [==============================] - 11s 1ms/step
Test loss: 1.4179688464164735
Test accuracy: 0.7568
Saved the model to disk


In [0]:
model.save_weights("Yolo_Basic_model2.h5")
print("Saved the model to disk")
from google.colab import files

files.download('Yolo_Basic_model.h5')

Saved the model to disk


FileNotFoundError: ignored